In [1]:
import json
import torch, torch.nn as nn
from torch import nn, optim, tensor
from torch.autograd import Variable
import random

filepath = ".../bnc_spoken_output_flattened.json"

    input_data = json.load(infile)
    text = input_data[:10]

# print(text)


seq_len = len(text)
batch_size = 1
embedding_size = 1
hidden_size = 1

# get max sequence length, then pad every sentence
max_len = max(len(sentence) for sentence in input_data)

vocab = set()

for sentence in input_data[0:10]:
    vocab = vocab.union(sentence)

vocab.add("<unk>")

vocab = list(vocab)
vocab_size = len(vocab)
output_size = vocab_size
word_ids = {word: id_ for id_, word in enumerate(vocab)}
ids_word = {id_: word for word, id_ in word_ids.items()}
'''
# text = input_data[random.randint(0, vocab_size - 1)]
indices = [word_ids[word] for word in text]
input_val = torch.eye(vocab_size)[indices].unsqueeze(1)

bi_rnn = torch.nn.RNN(
    input_size=vocab_size, hidden_size=hidden_size, num_layers=1, batch_first=False, bidirectional=True)

bi_output, bi_hidden = bi_rnn(input_val)

# stagger
forward_output, backward_output = bi_output[:-2, :, :hidden_size], bi_output[2:, :, hidden_size:]
staggered_output = torch.cat((forward_output, backward_output), dim=-1)

linear = nn.Linear(hidden_size * 2, output_size)

# only predict on words
labels = torch.LongTensor(indices[1:-1])

# for language models, use cross-entropy :)
loss = nn.CrossEntropyLoss()
output = loss(linear(staggered_output.squeeze(1)), labels)

print(output)
'''

'\n# text = input_data[random.randint(0, vocab_size - 1)]\nindices = [word_ids[word] for word in text]\ninput_val = torch.eye(vocab_size)[indices].unsqueeze(1)\n\nbi_rnn = torch.nn.RNN(\n    input_size=vocab_size, hidden_size=hidden_size, num_layers=1, batch_first=False, bidirectional=True)\n\nbi_output, bi_hidden = bi_rnn(input_val)\n\n# stagger\nforward_output, backward_output = bi_output[:-2, :, :hidden_size], bi_output[2:, :, hidden_size:]\nstaggered_output = torch.cat((forward_output, backward_output), dim=-1)\n\nlinear = nn.Linear(hidden_size * 2, output_size)\n\n# only predict on words\nlabels = torch.LongTensor(indices[1:-1])\n\n# for language models, use cross-entropy :)\nloss = nn.CrossEntropyLoss()\noutput = loss(linear(staggered_output.squeeze(1)), labels)\n\nprint(output)\n'

In [2]:
class biRNNLM(nn.Module):

    def __init__(self, vocab, hidden_size, freeze_embeddings=True,
                 recurrent_activation="tanh", recurrent_layers=1, recurrent_bidirectional=False):
        super(biRNNLM, self).__init__()
        
        self.vocab = vocab
        
        self.embedding = nn.Embedding(vocab_size, 10)
        
        # The embeddings go into an RNN layer with recurrent_dim units
        self.bi_rnn = torch.nn.RNN(input_size=10, hidden_size=hidden_size, num_layers=1, batch_first=False, bidirectional=True)

        self.linear = nn.Linear(hidden_size * 2, output_size)
        
        self.loss_function = nn.CrossEntropyLoss()
        
    def forward(self, text, seq_lengths=max_len):
        unk_index = word_ids.get('<unk>')  # fallback index
        indices = torch.LongTensor([
        word_ids[word] if word in word_ids else unk_index
        for word in text
        ]).unsqueeze(1)
        word_embeddings = self.embedding(indices)
        
        bi_output, bi_hidden = self.bi_rnn(word_embeddings)

        # stagger
        forward_output, backward_output = bi_output[:-2, :, :hidden_size], bi_output[2:, :, hidden_size:]
        staggered_output = torch.cat((forward_output, backward_output), dim=-1)

        # only predict on words
        labels = torch.LongTensor(indices[1:-1])

        # for language models, use cross-entropy :)

        output = self.loss_function(self.linear(staggered_output.squeeze(1)), labels.view(-1))
        
        return output

In [ ]:
model = biRNNLM(vocab=vocab, hidden_size=hidden_size)
optimizer = optim.Adam(model.parameters())

def train(model=model, optimizer=optimizer, epochs=10, print_every=1,
          validation_data=None):
    current_loss = 0.0
    minibatches_per_log = len(input_data)
    for epoch in range(epochs):
        # Within each epoch, iterate over the data in mini-batches
        # Note the use of *datapoint_list for generality, whether or not there are offsets
        for text in input_data:
            
            model.zero_grad()

            loss = model(text)  # already returns the loss
            loss.backward()
            optimizer.step()

            if print_every is not None:
                current_loss += loss.item()

            # Log performance
        if print_every is not None and (epoch + 1) % print_every == 0:
            log_message = ('| epoch {:3d} | train loss {:6.3f} |'
                             .format(epoch + 1, current_loss/minibatches_per_log))
            if validation_data is not None:
                validation_performance = test(model, validation_data)
                log_message += 'valid loss {loss:6.3f} |'.format(**validation_performance)
            print(log_message)

            # Reset trackers after logging
            current_loss = 0.0
            model.train()
train(model, optimizer, 10, 1, None)